# Оптимизационные задачи в ритейле

Привет, Habr! На связи отдел аналитики данных X5 Tech.

Сегодня мы поговорим об очень интересном разделе прикладной математики — оптимизации.
Так как данная тема достаточно обширна, то помимо данной статьи будет ещё 2:

    1) Сравнение open-source солверов на примере задачи ритейла [статья](http://)
    2) Решение модельной задачи ценообразования оптимизаторами в различных постановках [статья](http://)


Цель данной статьи — рассказать про задачи, которые решаются методами оптимизации
и продемонстрировать применение пакета [__Pyomo__](http://www.pyomo.org/) для решения модельной задачи ценообразования.


## Примеры задач


Практически каждый человек ежедневно решает оптимизационные задачи даже не задумываясь об этом. Пара примеров:

**Закупка.**
Как правило, мы хотим минимизировать наши расходы для приобретения необходимых товаров, но при условии, чтобы эти товары были максимально полезны.
Полезность здесь у каждого своя:
для одних она определяется количеством растительных жиров,
для других — минимальной суммарной стоимостью корзины,
для третьих — наличием привычных товаров, и тд.

**Отпуск.**
Во время ограниченного отпуска мы хотим распределить свой маршрут так,
чтобы посетить всё, что запланировали с минимальными временными затратами на дорогу, и при этом не забыть позагорать на пляже.

Бизнес требования часто приводят к задаче многокритериальной оптимизации и задаче оптимального управления.
В таких постановках решения, найденные методами оптимизации, чрезвычайно полезны для принятия решений.
Приведём для иллюстрации несколько задач из области ритейла.

**Ценообразование.**
Поиск оптимальной конфигурации цен с учетом ценового позиционирования, допустимых ценовых диапазонов
для каждого товара и набора бизнес-ограничений.
Цены должны максимизировать суммарный доход, а прибыль быть не ниже на заданного уровня.

**Оптимальное распределение маркетингового бюджета.**
Реализовать максимально эффективно выделенный на маркетинговые активности бюджет.
Есть несколько каналов для рекламных акций, цель — максимально выгодно инвестировать бюджет,
чтобы суммарный доход со всех коммуникаций был максимален.
Также необходимо учесть бизнес ограничения на нагрузку каждого канала + частоту взаимодействия.

**Планирование ассортимента.**
Подобрать ассортимент товаров так, чтобы максимизировать оборот с учетом полочного пространства и других характеристик магазина.

**Закупка товаров.**
Задача — распределить бюджет, выделенный на закупки, для поддержания товарооборота, достаточного уровня сервиса,
и при этом достигнуть определенных финансовых показателей под выделенный бюджет на закупки.

**Логистика.**
Задача — найти оптимальный график доставки продуктов с учётом вместимости грузовиков, затрат на логистику и тд.


## Общая постановка задачи и её разновидности


Прежде всего рассмотрим постановку задачи в общем виде:

$x$ - вектор размерности $n$, $x \in X$ - допустимое множество значений этих переменных.

$f(x) \to \min(\max)$, $f(\cdot)$ - целевая функция

$g_i(x) \leqslant 0, \ i=1..m$ - ограничения вида неравенств

$h_i(x) = 0, \ j=1..k$ - ограничения вида равенств

Исходя из практики можно разложить данную постановку на несколько классов в зависимости от вида целевой функции, ограничений и $X$:

* __Безусловная оптимизация__ $g_i(x), h_j(x)$ - отсутствуют, $X = \mathbb{R}^n$;

* __LP__ (linear programming) - линейное программирование. $f(x), g_i(x), h_j(x)$ - линейные функции, $X = \mathbb{R}_+^n$;

* __MILP__ (mixed integer linear programming) - смешанное целочисленное линейное программирование, это задача LP в которой только часть переменных являются целочисленными;

* __NLP__ (nonlinear programming) - нелинейное программирование, возникает когда хотя бы одна из функций $f(x),\ g_i(x),\ h_j(x)$ нелинейна;

* __MINLP__ (mixed integer nonlinear programming) - смешанное целочисленное нелинейное программирование,
возникает как и в MILP, когда часть переменных принимает целочисленные значения;

__NLP__ в свою очередь можно подробить еще на множество разных классов в зависимости от вида нелинейности и выпуклости.


## Оптимизация модельной задачи ценообразования

Предположим, что для товара $i$ известно значение эластичности $E_{i}$, а спрос задаётся следующей зависимостью:

\begin{equation}
Q_{i}(P_{i}) = Q_{0, i} \exp\bigg(E_{i} \cdot \bigg(\frac{P_{i}}{P_{0, i}} - 1\bigg)\bigg).
\end{equation}

Введём обозначения:
* $n$ - количество товаров,
* $C_{i}$ - себестоимость i-го товара,
* $Q_{i}, Q_{0, i}$ - спрос по новой $P_{i}$ и текущей $P_{0, i}$ ценам, соответственно,
* $x_{i}=\cfrac{P_{i}}{P_{0, i}}$.

Задача — найти такой набор новых цен, чтобы:
 * максимизировать оборот со всех товаров
 * общая прибыль осталась на прежнем уровне
 * цены лежали в заданных границах.

Тогда оптимизационную задачу можно записать следующей системой:

\begin{cases}
\tag{1}
   \sum_{i=1}^{n} P_i \cdot Q_i(x_i) \to \max_{x},\\
   \\
   \sum_{i=1}^{n} (P_i - C_i) \cdot Q_i(x_i) \geqslant M_0,\\
   \\
   x_i \in [x_i^l, x_i^u], \ i=1..n\\
\end{cases}

### Реализация модели в PYOMO и scipy

In [8]:
# from data_generator.data_generator import generate_simple_data
# generate_simple_data(10).round(3).rename(columns={'plu':'sku',
#                                                   'P': 'P0',
#                                                   'Q': 'Q0',
#                                                   'x_lower': 'x_l',
#                                                   'x_upper': 'x_u',
#                                                   'x_init': 'x',
#                                                   })#.to_markdown()

Сгенерируем набор данных для солверов:

```python
data = generate_base_data(N)
```

<details>
<summary> Пример данных для NLP постановки </summary>
<img src="images/a11_data.png" width="300" align="left"/>
</details>

Для каждого солвера необходимо по своим правилам задать следующие методы базового класса:

Описание классов здесь займет достаточно много места, детали можно найти [тут, код-github](https://github.com/mbudylin/OptimizersArticle/blob/main/optimizers/optimizers.py),

* init_objective - задание целевой функции,
* add_con_mrg - ограничений и метод,
* init_variables - задание переменных и границ.
* solve - поиск оптимального решения

<details>
<div><pre><code class="python">
class OptimizationModel(abc.ABC):
    """
    Базовый класс для моделей оптимизации
    """
    def init_variables(self):
        ...
    def init_objective(self):
        ...
    def init_constraints(self):
        ...
    def solve(self):
        ...
</code></pre><p></p></div>
</details>

#### init_variables

#### init_objective

init_objective:
* Для PYOMO передать в класс Objective формулу оборота:

```python
# pyomo
def init_objective(self):
    objective = sum(self.P[i] * self.model.x[i] * self.Q[i] * self._el(i) for i in range(self.N))
    self.model.obj = pyo.Objective(expr=objective, sense=pyo.maximize)
```

Для scipy передать в scipy.optimize.minimize функцию пересчёта оборота (со знаком минус, чтобы минимизация изменилась на максимизацию:

```python
# scipy
def init_objective(self):
    def objective(x):
        x_ = x[self.plu_line_idx[self.plu_idx]]
        f = -sum(self.P * x_ * self.Q * self._el(self.E, x_))
        return f / self.k

    self.obj = objective

```

#### init_constraints

#### solve

## Расчёт


Чтобы запустить расчёты, нужно выполнить команду:

```console
python runner.py -m pyomo
python runner.py -m scipy
```

### Сравнение расчётов

```console
python runner.py -m compare
```

Результаты замеров представлены ниже на графиках.

## Заключение

В данной статье мы рассмотрели, какие задачи оптимизации бывают на практике, и научились решать
 их с помощью пакета __PYOMO__.

В следующей стать более детально обсудим другие пакеты оптимизации для python и их различия.

Над статьёй работали Михаил Будылин, Антон Денисов, Михаил Дубровский.
